## Method used to calculate global protected areas and OECM coverage

The protected area coverage is calculated (from WDPA [methodolgy](https://www.protectedplanet.net/en/resources/calculating-protected-area-coverage)):

1. Start with the latest WDPA monthly release
2. The WDPA is filtered to exclude records with the characteristics listed in Section 2
3. A buffer is created around protected areas reported as points using their Reported Area. There are important caveats associated with this method, some of which are explored by Visconti et al. 2013. Buffering points can underestimate or overestimate protected area coverage as the circles created around points might cover areas where protected areas do not exist (overestimation) or overlap with areas where other protected areas already exist (underestimation). It can also give inaccurate values for sites that are partly terrestrial and marine as the absence of boundaries make it difficult to predict which portion of a protected area is in the land or the sea.
4. Both polygon and buffered point layers are combined in a single layer
5. The layer above is flattened (dissolved) – to eliminate overlaps between designations and avoid double counting.
6. The global protected areas flat layer is intersected with a base map of the world (see Section 3)
7. The intersected flat layer is converted to Mollweide (an equal area projection) and the area of each polygon is calculated, in km2.
8. Calculated areas are summed by land, marine and Areas Beyond National Jurisdiction (ABNJ). Marine and coastal area are those outlined in the Economic Exclusion Zones dataset (see Section 3 above). ABNJ constitute international waters outside the 200 nautical mile limits of national jurisdiction.
9. The terrestrial protected area coverage is calculated by dividing the total area of terrestrial protected areas by total global terrestrial area excluding Antarctica. ABNJ protected area coverage is calculated by selecting areas where ISO3 = 'ABNJ'. Marine and coastal protected area coverage is total global protected areas flat coverage - (ABNJ Area + Land Area).

### Set up

In [14]:
import os
import geopandas as gpd
import pandas as pd
from datetime import datetime

In [ ]:
path_in = "/Users/sofia/Documents/Repos/skytruth_30x30/data/raw"
path_out = "/Users/sofia/Documents/Repos/skytruth_30x30/data/processed"

### 1. Download and import last release of [MPA](https://www.protectedplanet.net/en/thematic-areas/marine-protected-areas): Sept 2023

In [ ]:
poly1 = gpd.read_file(path_in + "/WDPA_WDOECM_Sep2023_Public_marine_shp/WDPA_WDOECM_Sep2023_Public_marine_shp_0/WDPA_WDOECM_Sep2023_Public_marine_shp-polygons.shp")
point1 = gpd.read_file(path_in + "/WDPA_WDOECM_Sep2023_Public_marine_shp/WDPA_WDOECM_Sep2023_Public_marine_shp_0/WDPA_WDOECM_Sep2023_Public_marine_shp-points.shp")
poly2 = gpd.read_file(path_in + "/WDPA_WDOECM_Sep2023_Public_marine_shp/WDPA_WDOECM_Sep2023_Public_marine_shp_1/WDPA_WDOECM_Sep2023_Public_marine_shp-polygons.shp")
point2 = gpd.read_file(path_in + "/WDPA_WDOECM_Sep2023_Public_marine_shp/WDPA_WDOECM_Sep2023_Public_marine_shp_1/WDPA_WDOECM_Sep2023_Public_marine_shp-points.shp")
poly3 = gpd.read_file(path_in + "/WDPA_WDOECM_Sep2023_Public_marine_shp/WDPA_WDOECM_Sep2023_Public_marine_shp_2/WDPA_WDOECM_Sep2023_Public_marine_shp-polygons.shp")
point3 = gpd.read_file(path_in + "/WDPA_WDOECM_Sep2023_Public_marine_shp/WDPA_WDOECM_Sep2023_Public_marine_shp_2/WDPA_WDOECM_Sep2023_Public_marine_shp-points.shp")


In [ ]:
print(len(poly1))
print(len(point1))
print(len(poly2))
print(len(point2))
print(len(poly3))
print(len(point3))

### 2. Filter WDPA to exclude records:
- "Non Reported" protected areas (methodology recommends to remove also "Proposed" but we keep it for future projections)
- MAB (Note: MAB sites reported as OECMs are included in coverage analyses)
- Sites submitted as points with no reported area

In [ ]:
dataframes = [poly1, point1, poly2, point2, poly3, point3]

for i, df in enumerate(dataframes):
    # Remove rows where 'status' is equal to 'Not Reported'
    df = df[df['STATUS'] != 'Not Reported']
    
    # Remove rows where 'DESIG' contains 'MAB'
    df = df[~df['DESIG_ENG'].str.contains('MAB', case=False)]
    
    # Check if the dataframe is one of point1, point2, or point3
    if i in [1, 3, 5]:
        # Remove rows where reported area is 0
        df = df[(df['REP_AREA'] != 0)]
    
    # Update the original dataframes in the list
    dataframes[i] = df



In [ ]:
print(len(dataframes[0]))
print(len(dataframes[1]))
print(len(dataframes[2]))
print(len(dataframes[3]))
print(len(dataframes[4]))
print(len(dataframes[5]))

### 3. Create buffers around points based on reported area

In [ ]:
# Calculate radius based on REP_AREA
def calculate_radius(rep_area):
    return (rep_area / 3.14159265358979323846) ** 0.5

# Iterate through the list and process the desired dataframes
for idx in [1, 3, 5]:
    # Get the dataframe at the specified index
    gdf = dataframes[idx]

    # Reproject in Mollweide
    gdf = gdf.to_crs('ESRI:54009')

    # Transform the reported area from square kilometers to square meters
    gdf['REP_AREA_m'] = gdf['REP_AREA'] * 1000000

    # Create the "radius" column by applying the calculate_radius function to the "REP_AREA" column
    gdf['radius'] = gdf['REP_AREA_m'].apply(calculate_radius)

    # Create buffers around the points using the "radius" column
    gdf_buffered = gdf.copy()
    gdf_buffered['geometry'] = gdf.apply(lambda row: row.geometry.buffer(row['radius']), axis=1)

    # Reproject back to WGS84
    gdf_buffered = gdf_buffered.to_crs('EPSG:4326')

    # Remove rows with invalid geometries
    gdf_buffered = gdf_buffered[gdf_buffered['geometry'].is_valid]
    
    # Update the original dataframe with the buffered data
    dataframes[idx] = gdf_buffered

### 4. Merge the 6 datasets (polygons and buffered points) in a single layer and segregate those that are "Proposed" and those that are OECM

In [ ]:
# Check that all of them have the same crs
first_crs = dataframes[0].crs
same_crs = all(gdf.crs == first_crs for gdf in dataframes[1:])
if same_crs:
    print("All gdf have the same crs:", first_crs)
else:
        print("gdf have different crs")

In [ ]:
# Check if geometries are valid
sum(merged_mpa.geometry.is_valid)

In [ ]:
# Save merged_mpa as a shapefile
merged_mpa.to_file(path_out + "/wdpa/merged_mpa.shp")

**Separate "Proposed"**

In [ ]:
# Select only the rows where 'STATUS' is equal to 'Proposed'
proposed = merged_mpa[merged_mpa['STATUS'] == 'Proposed']

# Select only the rows where 'STATUS' is different from 'Proposed'
protected = merged_mpa[merged_mpa['STATUS'] != 'Proposed']

In [ ]:
# Save the two dataframes as shapefiles
proposed.to_file(path_out + "/wdpa/proposed.shp")
protected.to_file(path_out + "/wdpa/protected.shp")

In [ ]:
print(len(proposed))
print(len(protected))

### 5. Dissolve intersecting polygons by relevant fields

In [ ]:
protected.columns

In [ ]:
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/protected.shp -filter 'STATUS_YR<=2000' -dissolve2 fields=PA_DEF, PARENT_ISO -explode -proj +proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs -each AREA=this.area -o /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/timeseries/protected_dissolved_2000.shp

In [ ]:
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/protected.shp -filter 'STATUS_YR<=2001' -dissolve2 fields=PA_DEF, PARENT_ISO -explode -proj +proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs -each AREA=this.area -o /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/timeseries/protected_dissolved_2001.shp

In [ ]:
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/protected.shp -filter 'STATUS_YR<=2002' -dissolve2 fields=PA_DEF, PARENT_ISO -explode -proj +proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs -each AREA=this.area -o /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/timeseries/protected_dissolved_2002.shp

In [ ]:
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/protected.shp -filter 'STATUS_YR<=2003' -dissolve2 fields=PA_DEF, PARENT_ISO -explode -proj +proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs -each AREA=this.area -o /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/timeseries/protected_dissolved_2003.shp

In [ ]:
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/protected.shp -filter 'STATUS_YR<=2004' -dissolve2 fields=PA_DEF, PARENT_ISO -explode -proj +proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs -each AREA=this.area -o /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/timeseries/protected_dissolved_2004.shp

In [ ]:
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/protected.shp -filter 'STATUS_YR<=2005' -dissolve2 fields=PA_DEF, PARENT_ISO -explode -proj +proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs -each AREA=this.area -o /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/timeseries/protected_dissolved_2005.shp

In [ ]:
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/protected.shp -filter 'STATUS_YR<=2006' -dissolve2 fields=PA_DEF, PARENT_ISO -explode -proj +proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs -each AREA=this.area -o /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/timeseries/protected_dissolved_2006.shp

In [ ]:
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/protected.shp -filter 'STATUS_YR<=2007' -dissolve2 fields=PA_DEF, PARENT_ISO -explode -proj +proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs -each AREA=this.area -o /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/timeseries/protected_dissolved_2007.shp

In [ ]:
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/protected.shp -filter 'STATUS_YR<=2008' -dissolve2 fields=PA_DEF, PARENT_ISO -explode -proj +proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs -each AREA=this.area -o /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/timeseries/protected_dissolved_2008.shp

In [ ]:
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/protected.shp -filter 'STATUS_YR<=2009' -dissolve2 fields=PA_DEF, PARENT_ISO -explode -proj +proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs -each AREA=this.area -o /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/timeseries/protected_dissolved_2009.shp

In [ ]:
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/protected.shp -filter 'STATUS_YR<=2010' -dissolve2 fields=PA_DEF, PARENT_ISO -explode -proj +proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs -each AREA=this.area -o /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/timeseries/protected_dissolved_2010.shp

In [ ]:
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/protected.shp -filter 'STATUS_YR<=2011' -dissolve2 fields=PA_DEF, PARENT_ISO -explode -proj +proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs -each AREA=this.area -o /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/timeseries/protected_dissolved_2011.shp

In [ ]:
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/protected.shp -filter 'STATUS_YR<=2012' -dissolve2 fields=PA_DEF, PARENT_ISO -explode -proj +proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs -each AREA=this.area -o /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/timeseries/protected_dissolved_2012.shp

In [ ]:
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/protected.shp -filter 'STATUS_YR<=2013' -dissolve2 fields=PA_DEF, PARENT_ISO -explode -proj +proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs -each AREA=this.area -o /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/timeseries/protected_dissolved_2013.shp

In [ ]:
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/protected.shp -filter 'STATUS_YR<=2014' -dissolve2 fields=PA_DEF, PARENT_ISO -explode -proj +proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs -each AREA=this.area -o /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/timeseries/protected_dissolved_2014.shp

In [ ]:
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/protected.shp -filter 'STATUS_YR<=2015' -dissolve2 fields=PA_DEF, PARENT_ISO -explode -proj +proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs -each AREA=this.area -o /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/timeseries/protected_dissolved_2015.shp

In [ ]:
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/protected.shp -filter 'STATUS_YR<=2016' -dissolve2 fields=PA_DEF, PARENT_ISO -explode -proj +proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs -each AREA=this.area -o /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/timeseries/protected_dissolved_2016.shp

In [ ]:
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/protected.shp -filter 'STATUS_YR<=2017' -dissolve2 fields=PA_DEF, PARENT_ISO -explode -proj +proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs -each AREA=this.area -o /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/timeseries/protected_dissolved_2017.shp

In [ ]:
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/protected.shp -filter 'STATUS_YR<=2018' -dissolve2 fields=PA_DEF, PARENT_ISO -explode -proj +proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs -each AREA=this.area -o /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/timeseries/protected_dissolved_2018.shp

In [ ]:
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/protected.shp -filter 'STATUS_YR<=2019' -dissolve2 fields=PA_DEF, PARENT_ISO -explode -proj +proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs -each AREA=this.area -o /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/timeseries/protected_dissolved_2019.shp

In [ ]:
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/protected.shp -filter 'STATUS_YR<=2020' -dissolve2 fields=PA_DEF, PARENT_ISO -explode -proj +proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs -each AREA=this.area -o /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/timeseries/protected_dissolved_2020.shp

In [ ]:
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/protected.shp -filter 'STATUS_YR<=2021' -dissolve2 fields=PA_DEF, PARENT_ISO -explode -proj +proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs -each AREA=this.area -o /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/timeseries/protected_dissolved_2021.shp

In [ ]:
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/protected.shp -filter 'STATUS_YR<=2022' -dissolve2 fields=PA_DEF, PARENT_ISO -explode -proj +proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs -each AREA=this.area -o /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/timeseries/protected_dissolved_2022.shp

In [ ]:
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/protected.shp -filter 'STATUS_YR<=2023' -dissolve2 fields=PA_DEF, PARENT_ISO -explode -proj +proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs -each AREA=this.area -o /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/wdpa/timeseries/protected_dissolved_2023.shp

### Calculate coverage statistics

**Countries per PARENT_ISO**

In [52]:
p2023 = gpd.read_file(path_out + "/wdpa/timeseries/protected_dissolved_2023.shp")

In [53]:
p2023['PARENT_ISO'].unique()

array(['ATG', 'BRB', 'BRA', 'GBR', 'CHL', 'COL', 'CRI', 'DOM', 'ECU',
       'JAM', 'NLD', 'PER', 'PAN', 'SUR', 'VEN', 'USA', 'AUS', 'CAN',
       'FRA', 'IRN', 'JPN', 'KEN', 'KOR', 'MYS', 'MRT', 'MOZ', 'NOR',
       'PHL', 'POL', 'SAU', 'SEN', 'SWE', 'THA', 'TUN', 'CMR', 'IDN',
       'MUS', 'PRT', 'SYC', 'ISL', 'NZL', 'EST', 'GEO', 'UKR', 'MEX',
       'BHS', 'BLZ', 'GMB', 'MDG', 'HRV', 'FJI', 'LKA', 'ARG', 'ZAF',
       'PNG', 'TON', 'PLW', 'COK', 'BGD', 'AGO', 'ALB', 'DNK', 'ITA',
       'PAK', 'FIN', 'VNM', 'MMR', 'CHN', 'SGP', 'DEU', 'ROU', 'EGY',
       'SLB', 'VUT', 'BGR', 'MAR', 'MLT', 'DMA', 'LCA', 'OMN', 'GTM',
       'NIC', 'TTO', 'WSM', 'TZA', 'GRC', 'LBN', 'CUB', 'ISR', 'GRD',
       'VCT', 'BRN', 'ESP', 'JOR', 'ARE', 'HND', 'GNQ', 'KNA', 'LTU',
       'GNB', 'NGA', 'LVA', 'GUY', 'KAZ', 'BEL', 'GIN', 'IRL', 'RUS',
       'KHM', 'QAT', 'GAB', 'MDV', 'AZE', 'NAM', 'TUR', 'CPV', 'COG',
       'TUV', 'MCO', 'TKM', 'SVN', 'SLE', 'KIR', 'COM', 'NIU', 'FSM',
       'GHA', 'IOT',

In [19]:
folder_path = path_out + '/wdpa/timeseries'
years_range = range(2000, 2024)  # Range of years from 2000 to 2023

# Create an empty list to store the results
results_list = []

for year in years_range:
    filename = f'protected_dissolved_{year}.shp'
    file_path = os.path.join(folder_path, filename)
    
    if os.path.exists(file_path):
        gdf = gpd.read_file(file_path)
        grouped = gdf.groupby('PARENT_ISO')['AREA'].sum().reset_index()
        
        # Create columns
        grouped['year'] = year
        grouped['protection_type'] = 'MPA+OECM'
        grouped.rename(columns={'PARENT_ISO': 'location_id', 'AREA': 'cumsum_area'}, inplace=True)
        
        # Append the result to the list
        results_list.append(grouped)

# Concatenate the list of results into a single DataFrame
final_df = pd.concat(results_list, ignore_index=True)


In [22]:
# Convert area to km2
final_df['cumsum_area'] =final_df['cumsum_area']/1000000

**Global**

In [41]:
# Calculate global per year and append it
glob_df = final_df.groupby(['year', 'protection_type'])['cumsum_area'].sum().reset_index()

glob_df['location_id'] = 'GLOB'

final_df2 = pd.concat([final_df, glob_df], ignore_index=True)
final_df2

,location_id,cumsum_area,year,protection_type
0,ABNJ,594174.66,2000,MPA+OECM
1,AGO,0.42,2000,MPA+OECM
2,ALB,103.05,2000,MPA+OECM
3,ARE,78.52,2000,MPA+OECM
4,ARG,6155.67,2000,MPA+OECM
...,...,...,...,...
3571,GLOB,28125365.96,2019,MPA+OECM
3572,GLOB,29624663.84,2020,MPA+OECM
3573,GLOB,29739178.77,2021,MPA+OECM
3574,GLOB,29910678.77,2022,MPA+OECM


In [42]:
# Reorder the columns and add column last updated
final_df2 = final_df2[['location_id', 'year', 'protection_type', 'cumsum_area']]

current_date = datetime.now().strftime('%Y-%m-%d')

final_df2 = final_df2.copy()
final_df2['last_updated'] = current_date

**Regions**

In [47]:
# List of dictionaries for data in Region_ISO3_PP.txt (list of regions used in the Protected Planet database)
regions_data = [
    {
        'region_iso': 'AS',
        'region_name': 'Asia & Pacific',
        'country_iso_3s': [
            "AFG", "ASM", "AUS", "BGD", "BRN", "BTN", "CCK", "CHN", "COK", "CXR", "FJI", "FSM", "GUM", "HKG", "IDN",
            "IND", "IOT", "IRN", "JPN", "KHM", "KIR", "KOR", "LAO", "LKA", "MAC", "MDV", "MHL", "MMR", "MNG", "MNP",
            "MYS", "NCL", "NFK", "NIU", "NPL", "NRU", "NZL", "PAK", "PCN", "PHL", "PLW", "PNG", "PRK", "PYF", "SGP",
            "SLB", "THA", "TKL", "TLS", "TON", "TUV", "TWN", "VNM", "VUT", "WLF", "WSM"
        ]
    },
    {
        'region_iso': 'AF',
        'region_name': 'Africa',
        'country_iso_3s': [
            "AGO", "BDI", "BEN", "BFA", "BWA", "CAF", "CIV", "CMR", "COD", "COG", "COM", "CPV", "DJI", "DZA", "EGY",
            "ERI", "ESH", "ETH", "GAB", "GHA", "GIN", "GMB", "GNB", "GNQ", "KEN", "LBR", "LBY", "LSO", "MAR", "MDG",
            "MLI", "MOZ", "MRT", "MUS", "MWI", "MYT", "NAM", "NER", "NGA", "REU", "RWA", "SDN", "SEN", "SHN", "SLE",
            "SOM", "SSD", "STP", "SWZ", "SYC", "TCD", "TGO", "TUN", "TZA", "UGA", "ZAF", "ZMB", "ZWE"
        ]
    },
    {
        'region_iso': 'EU',
        'region_name': 'Europe',
        'country_iso_3s': [
            "ALA", "ALB", "AND", "ARM", "AUT", "AZE", "BEL", "BGR", "BIH", "BLR", "CHE", "CYP", "CZE", "DEU", "DNK",
            "ESP", "EST", "FIN", "FRA", "FRO", "GBR", "GEO", "GGY", "GIB", "GRC", "HRV", "HUN", "IMN", "IRL", "ISL",
            "ISR", "ITA", "JEY", "KAZ", "KGZ", "LIE", "LTU", "LUX", "LVA", "MCO", "MDA", "MKD", "MLT", "MNE", "NLD",
            "NOR", "POL", "PRT", "ROU", "RUS", "SJM", "SMR", "SRB", "SVK", "SVN", "SWE", "TJK", "TKM", "TUR", "UKR",
            "UZB", "VAT"
        ]
    },
    {
        'region_iso': 'SA',
        'region_name': 'Latin America & Caribbean',
        'country_iso_3s': [
            "ABW", "AIA", "ARG", "ATG", "BES", "BHS", "BLM", "BLZ", "BMU", "BOL", "BRA", "BRB", "CHL", "COL", "CRI",
            "CUB", "CUW", "CYM", "DMA", "DOM", "ECU", "FLK", "GLP", "GRD", "GTM", "GUF", "GUY", "HND", "HTI", "JAM",
            "KNA", "LCA", "MAF", "MEX", "MSR", "MTQ", "NIC", "PAN", "PER", "PRI", "PRY", "SLV", "SUR", "SXM", "TCA",
            "TTO", "UMI", "URY", "VCT", "VEN", "VGB", "VIR"
        ]
    },
    {
        'region_iso': 'PO',
        'region_name': 'Polar',
        'country_iso_3s': [
            "ATF", "BVT", "GRL", "HMD", "SGS"
        ]
    },
    {
        'region_iso': 'NA',
        'region_name': 'North America',
        'country_iso_3s': [
            "CAN", "SPM", "USA"
        ]
    },
    {
        'region_iso': 'GL',
        'region_name': 'Global',
        'country_iso_3s': []
    },
    {
        'region_iso': 'WA',
        'region_name': 'West Asia',
        'country_iso_3s': [
            "ARE", "BHR", "IRQ", "JOR", "KWT", "LBN", "OMN", "PSE", "QAT", "SAU", "SYR", "YEM"
        ]
    }
]

# Convert the region data to a dictionary that maps each country to its region name
country_to_region = {}
for region in regions_data:
    for country in region['country_iso_3s']:
        country_to_region[country] = region['region_name']

In [48]:
final_df['region'] = final_df['location_id'].map(country_to_region)
final_df

,location_id,cumsum_area,year,protection_type,region
0,ABNJ,594174.66,2000,MPA+OECM,NaN
1,AGO,0.42,2000,MPA+OECM,Africa
2,ALB,103.05,2000,MPA+OECM,Europe
3,ARE,78.52,2000,MPA+OECM,West Asia
4,ARG,6155.67,2000,MPA+OECM,Latin America & Caribbean
...,...,...,...,...,...
3547,VNM,5036.97,2023,MPA+OECM,Asia & Pacific
3548,VUT,83.83,2023,MPA+OECM,Asia & Pacific
3549,WSM,199.59,2023,MPA+OECM,Asia & Pacific
3550,YEM,4108.19,2023,MPA+OECM,West Asia


In [49]:
final_df[final_df.region.isnull()]

,location_id,cumsum_area,year,protection_type,region
0,ABNJ,594174.66,2000,MPA+OECM,NaN
5,ATA,3594.42,2000,MPA+OECM,NaN
37,FIN;SWE,3541.14,2000,MPA+OECM,NaN
131,ABNJ,594174.66,2001,MPA+OECM,NaN
136,ATA,3594.42,2001,MPA+OECM,NaN
...,...,...,...,...,...
3397,ABNJ,2811451.69,2023,MPA+OECM,NaN
3402,ATA,3570.36,2023,MPA+OECM,NaN
3439,FIN;SWE,3541.14,2023,MPA+OECM,NaN
3442,FRA;ITA;MCO,87742.14,2023,MPA+OECM,NaN


In [55]:
final_df['location_id'].unique() 

array(['ABNJ', 'AGO', 'ALB', 'ARE', 'ARG', 'ATA', 'ATG', 'AUS', 'AZE',
       'BEL', 'BGD', 'BGR', 'BHR', 'BHS', 'BLZ', 'BRA', 'BRB', 'BRN',
       'CAN', 'CHL', 'CHN', 'COD', 'COG', 'COK', 'COL', 'CRI', 'CUB',
       'CYP', 'DEU', 'DMA', 'DNK', 'DOM', 'ECU', 'EGY', 'ESP', 'EST',
       'FIN', 'FIN;SWE', 'FJI', 'FRA', 'FSM', 'GBR', 'GEO', 'GHA', 'GIN',
       'GMB', 'GNB', 'GNQ', 'GRC', 'GRD', 'GTM', 'HND', 'HRV', 'IDN',
       'IRL', 'IRN', 'ISL', 'ISR', 'ITA', 'JAM', 'JPN', 'KAZ', 'KEN',
       'KHM', 'KIR', 'KNA', 'KOR', 'LBN', 'LBY', 'LCA', 'LKA', 'LTU',
       'LVA', 'MAR', 'MCO', 'MDG', 'MDV', 'MEX', 'MHL', 'MLT', 'MMR',
       'MNE', 'MOZ', 'MRT', 'MUS', 'MYS', 'NAM', 'NGA', 'NIC', 'NIU',
       'NLD', 'NOR', 'NZL', 'OMN', 'PAK', 'PAN', 'PER', 'PHL', 'PLW',
       'PNG', 'POL', 'PRT', 'ROU', 'RUS', 'SAU', 'SDN', 'SEN', 'SLB',
       'SLE', 'SUR', 'SVN', 'SWE', 'SYC', 'SYR', 'THA', 'TKM', 'TLS',
       'TON', 'TTO', 'TUN', 'TUR', 'TUV', 'TZA', 'UKR', 'USA', 'VCT',
       'VEN', '

In [44]:
final_df2.to_csv(path_out + '/tables/protected_area_coverage.csv', index=False)